In [1]:
import sys
import pandas as pd
import numpy as np
from common import commons 
home = commons.home
from log import Logger
import os

In [2]:
def cal_beta(beta_file,pos_file):
    betas = pd.read_csv(beta_file,sep='\s+',index_col=['TargetID'])
    mean_betas = pd.DataFrame(betas.mean(axis=1),columns=['beta'])
    mean_betas.index = betas.index
    pos = pd.read_csv(pos_file,sep='\s+',usecols=[0,2,3],index_col=0, header=None,skiprows=1,names=['id','chr','coordinate'])
    beta_pos = mean_betas.join(pos)
    return beta_pos

In [3]:
log_dir = home+'logs/'
logger = Logger.Logger(log_dir).get_logger()
beta_file = home+'data/AD_CpG/ROSMAP_arrayMethylation_imputed.tsv'
pos_file = home+'data/AD_CpG/ROSMAP_arrayMethylation_metaData.tsv'
all_sites_betas = cal_beta(beta_file,pos_file)

In [4]:
all_sites_betas

,beta,chr,coordinate
TargetID,,,
cg00000165,0.154822,1,91194674
cg00000363,0.131544,1,230560793
cg00000957,0.779174,1,5937253
cg00001349,0.893716,1,166958439
cg00001364,0.727746,1,214170376
cg00001446,0.843326,1,43831041
cg00001534,0.882839,1,51034865
cg00001583,0.073302,1,200011786
cg00001593,0.927707,1,170490434


In [5]:
type_name

NameError: name 'type_name' is not defined

In [11]:
pos_pvalue = 0.0002 ### 0.001 for amyloid, 0.0001 for cerad, 0.00001 for tangles,0.002 for cogdec, 0.0002 for gpath,
neg_pvalue = 0.4
sample_ratio_neg_to_pos = 10

In [12]:
type_name = commons.type_name  ## amyloid, cerad, tangles
with_cell_type = commons.with_cell_type ## with or without
all_sites_file = home+'data/AD_CpG/Rosmap_'+type_name+'_ewas_'+with_cell_type+'celltype.csv'
all_sites = pd.read_csv(all_sites_file,usecols=[1,2,3],header=None,skiprows=1,index_col=0,names=['id','beta_sign','pvalue'])
all_sites = all_sites.join(all_sites_betas).dropna()
all_sites.reset_index(inplace=True)
temp = pd.DataFrame()
temp['id'],temp['chr'],temp['coordinate'],temp['beta_sign'],temp['pvalue'],temp['beta'] = all_sites['id'],all_sites['chr'],all_sites['coordinate'],all_sites['beta_sign'],all_sites['pvalue'],all_sites['beta']
all_sites = temp
all_sites['chr'] = all_sites['chr'].astype('i8')
all_sites.sort_values(['pvalue'],inplace=True,ascending=True)
positive_sites = all_sites.query('pvalue<=@pos_pvalue')
positive_sites['label'] = np.where(positive_sites['beta_sign']>0,1,-1)
negative_sites = all_sites.query('pvalue>@neg_pvalue')
negative_sites['label'] = 0
negatives_sort_by_beta = negative_sites.sort_values(['beta'])

/home/ec2-user/anaconda3/envs/py3.6/lib/python3.6/site-packages/ipykernel/__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/ec2-user/anaconda3/envs/py3.6/lib/python3.6/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [13]:
positive_sites

,id,chr,coordinate,beta_sign,pvalue,beta,label
49010,cg25594100,7,4786943.0,5.615169,3.258135e-08,0.780079,1
129485,cg13390284,1,65531864.0,5.032855,6.748768e-07,0.128000,1
51592,cg11823178,8,41519399.0,5.000947,7.904325e-07,0.919399,1
50455,cg05066959,8,41519308.0,4.962907,9.532788e-07,0.865159,1
124441,cg08706567,1,43814983.0,4.784900,2.254117e-06,0.121573,1
75416,cg00931644,13,77461368.0,4.743859,2.738557e-06,0.232159,1
48365,cg22962123,7,27153605.0,4.712061,3.181293e-06,0.264542,1
4774,cg11839415,1,43814764.0,4.678513,3.722806e-06,0.183050,1
61024,cg23968456,10,73521631.0,4.610318,5.109414e-06,0.968990,1
44393,cg07180538,7,4786899.0,4.585698,5.722642e-06,0.868481,1


In [15]:
negative_sites

,id,chr,coordinate,beta_sign,pvalue,beta,label
115109,cg00404053,1,178313656.0,-0.842335,0.400002,0.363377,0
338943,cg12413346,15,43425285.0,0.842332,0.400004,0.401286,0
96801,cg20940164,17,48450551.0,0.842325,0.400008,0.007785,0
56172,cg00929860,10,73846448.0,0.842323,0.400009,0.594119,0
271370,cg02819347,10,120354844.0,0.842323,0.400009,0.106417,0
234705,cg03081478,7,2773127.0,0.842322,0.400010,0.304118,0
49194,cg26308704,7,158325586.0,0.842321,0.400010,0.795521,0
314728,cg22345432,12,120650215.0,0.842317,0.400012,0.891192,0
246540,cg21785750,7,142428317.0,0.842312,0.400015,0.809100,0
96531,cg20062401,17,12569089.0,0.842293,0.400026,0.011908,0


In [16]:
select_negs_list = []
hyper_sites = negatives_sort_by_beta.query('beta_sign>=0')
hypo_sites = negatives_sort_by_beta.query('beta_sign<0')
for beta,beta_sign in positive_sites[['beta','beta_sign']].values:
    tmp_sites = hyper_sites if beta_sign >=0 else hypo_sites
    neg_ix = tmp_sites['beta'].searchsorted(beta)[0]    
    negs = tmp_sites.iloc[neg_ix-int(sample_ratio_neg_to_pos/2):np.minimum(neg_ix+int(sample_ratio_neg_to_pos/2),len(negatives_sort_by_beta)),:]
    select_negs_list.extend(negs.values)
select_negs = pd.DataFrame(select_negs_list,columns=['id','chr','coordinate','beta_sign','pvalue','beta','label'])

In [17]:
win_path = home+'wins.txt'
pos_sites_with_winid, neg_sites_with_winid = commons.merge_with_feature_windows(win_path,positive_sites,select_negs)
all_sites_with_winid = pos_sites_with_winid.append(neg_sites_with_winid,ignore_index=True)
all_sites_with_winid.drop_duplicates(['id'],inplace=True)
all_sites_with_winid.sort_values(['chr','coordinate'],inplace=True) 

/home/ec2-user/anaconda3/envs/py3.6/lib/python3.6/site-packages/pandas/core/generic.py:5984: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/home/ec2-user/git/EnsembleCpG/code/common/commons.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data['chr'] = data['chr'].astype('i8')
/home/ec2-user/git/EnsembleCpG/code/common/commons.py:101: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  feature_wins.so

In [19]:
#export all features to file
if not os.path.exists(home+'data/AD_CpG/'+type_name+with_cell_type):
    os.mkdir(home+'data/AD_CpG/'+type_name+with_cell_type)
with pd.HDFStore(home+'data/AD_CpG/'+type_name+with_cell_type+'/all_sites_winid','w') as h5s:
    h5s['all_sites_winid'] = all_sites_with_winid       
all_sites_with_winid.to_csv(home+'data/AD_CpG/'+type_name+with_cell_type+'/all_sites_winid.csv',index=False)  
all_sites_with_winid['winid'].to_csv(home+'data/AD_CpG/'+type_name+with_cell_type+'/selected_pos_winid.csv',index=False)

In [20]:
all_sites_with_winid

,id,chr,coordinate,beta_sign,pvalue,beta,label,start,winid,end
126,cg24730157,1,984428.0,0.277121,0.781802,0.919376,0,984401,4923,984600
127,cg17034203,1,1047097.0,0.518990,0.603997,0.747278,0,1047001,5236,1047200
128,cg13362546,1,1102960.0,0.407516,0.683803,0.780087,0,1102801,5515,1103000
129,cg19237691,1,1141617.0,0.105006,0.916413,0.287499,0,1141601,5709,1141800
130,cg24163210,1,1713944.0,0.168467,0.866284,0.214674,0,1713801,8570,1714000
131,cg17740743,1,1986220.0,0.025678,0.979524,0.965001,0,1986201,9932,1986400
132,cg02850689,1,2391347.0,0.686815,0.492517,0.276659,0,2391201,11957,2391400
133,cg12929805,1,2399054.0,0.531922,0.595016,0.151235,0,2399001,11996,2399200
134,cg00980319,1,2560884.0,0.799026,0.424654,0.230109,0,2560801,12805,2561000
135,cg25875953,1,3001128.0,0.259029,0.795719,0.702873,0,3001001,15006,3001200


In [30]:
positives = commons.sample_weights(all_sites_with_winid,all_sites_with_winid['label'],factor=1.5)

In [21]:
##export winid with all 450k sites
type_name = commons.type_name  ## amyloid, cerad, tangles
with_cell_type = commons.with_cell_type ## with or without
all_sites_file = home+'data/AD_CpG/Rosmap_'+type_name+'_ewas_'+with_cell_type+'celltype.csv'
all_sites = pd.read_csv(all_sites_file,usecols=[1,2,3],header=None,skiprows=1,index_col=0,names=['id','beta_sign','pvalue'])
all_sites = all_sites.join(all_sites_betas).dropna()
all_sites.reset_index(inplace=True)
temp = pd.DataFrame()
temp['id'],temp['chr'],temp['coordinate'],temp['beta_sign'],temp['pvalue'],temp['beta'] = all_sites['id'],all_sites['chr'],all_sites['coordinate'],all_sites['beta_sign'],all_sites['pvalue'],all_sites['beta']
all_sites = temp
all_sites['chr'] = all_sites['chr'].astype('i8')
all_sites.sort_values(['pvalue'],inplace=True,ascending=True)

In [22]:
all_sites

,id,chr,coordinate,beta_sign,pvalue,beta
49010,cg25594100,7,4786943.0,5.615169,3.258135e-08,0.780079
129485,cg13390284,1,65531864.0,5.032855,6.748768e-07,0.128000
51592,cg11823178,8,41519399.0,5.000947,7.904325e-07,0.919399
50455,cg05066959,8,41519308.0,4.962907,9.532788e-07,0.865159
124441,cg08706567,1,43814983.0,4.784900,2.254117e-06,0.121573
75416,cg00931644,13,77461368.0,4.743859,2.738557e-06,0.232159
48365,cg22962123,7,27153605.0,4.712061,3.181293e-06,0.264542
4774,cg11839415,1,43814764.0,4.678513,3.722806e-06,0.183050
61024,cg23968456,10,73521631.0,4.610318,5.109414e-06,0.968990
44393,cg07180538,7,4786899.0,4.585698,5.722642e-06,0.868481


In [23]:
win_path = home+'wins.txt'
all_450k_sites_with_winid, __ = commons.merge_with_feature_windows(win_path,all_sites)
all_450k_sites_with_winid.drop(['beta_sign'],axis=1,inplace=True)

/home/ec2-user/anaconda3/envs/py3.6/lib/python3.6/site-packages/pandas/core/generic.py:5984: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/home/ec2-user/git/EnsembleCpG/code/common/commons.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data['chr'] = data['chr'].astype('i8')
/home/ec2-user/git/EnsembleCpG/code/common/commons.py:101: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  feature_wins.so

In [24]:
all_450k_sites_with_winid

,id,chr,coordinate,pvalue,beta,start,winid,end
0,cg13869341,1,15865.0,0.926372,0.877442,15801,80,16000
1,cg24669183,1,534242.0,0.943379,0.786274,534201,2672,534400
2,cg15560884,1,710097.0,0.465516,0.652625,710001,3551,710200
3,cg01014490,1,714177.0,0.853347,0.008987,714001,3571,714200
4,cg17505339,1,720865.0,0.052664,0.859844,720801,3605,721000
5,cg11954957,1,758829.0,0.160539,0.816452,758801,3795,759000
6,cg23803172,1,763119.0,0.452868,0.004858,763001,3816,763200
7,cg16736630,1,779995.0,0.128003,0.758187,779801,3900,780000
8,cg05898754,1,805102.0,0.671161,0.359355,805001,4026,805200
9,cg03128332,1,805338.0,0.112397,0.142779,805201,4027,805400


In [25]:
with pd.HDFStore(home+'data/AD_CpG/'+type_name+with_cell_type+'/all_450k_sites_winid','w') as h5s:
    h5s['all_450k_sites_winid'] = all_450k_sites_with_winid      
all_450k_sites_with_winid.to_csv(home+'data/AD_CpG/'+type_name+with_cell_type+'/all_450k_sites_winid.csv',index=False)  
all_450k_sites_with_winid['winid'].to_csv(home+'data/AD_CpG/'+type_name+with_cell_type+'/selected_450k_pos_winid.csv',index=False)